In [130]:
# ============================================
# 📊 SaaS Event Data — Advanced Preprocessing
# ============================================

import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from category_encoders.target_encoder import TargetEncoder

# === Utility ===
def memory_usage_mb(df):
    return df.memory_usage(deep=True).sum() / 1024**2

In [131]:
input_path = "events_data.csv"
df = pd.read_csv(input_path, low_memory=False)
print(f"Loaded {len(df):,} rows and {df.shape[1]} columns")
print(f"Initial memory usage: {memory_usage_mb(df):.2f} MB")

Loaded 134,911 rows and 27 columns
Initial memory usage: 145.28 MB


In [132]:
# === Drop irrelevant identifiers ===
drop_cols = [
    "id", "trackingId", "sessionId", "userId", "ip",
    "customData", "expiresAt"
]
df = df.drop(columns=[c for c in drop_cols if c in df.columns], errors="ignore")

In [133]:
# === Drop duplicates ===
df = df.drop_duplicates()
print(f"Removed {len(df) - len(df.drop_duplicates())} duplicate rows")

Removed 0 duplicate rows


In [134]:
print(df.isnull().sum())

eventType                0
url                      0
screenResolution         0
browser                  0
language                 0
country              54263
region               54263
city                 54263
timezone              1858
clickX               68629
clickY               68629
scrollPercent        95849
durationMs           79136
domLoadTime          97893
fullLoadTime         97893
ttfb                 97893
downlink            101828
effectiveType       101828
rtt                 101828
errorMessage        109467
errorSource         109475
errorLine           109467
errorColumn         109467
dtype: int64


In [135]:
# === Clean strings & fill missing ===
df = df.apply(lambda col: col.str.strip() if col.dtype == "object" else col)

cat_fill = {
    "country": "unknown",
    "region": "unknown",
    "city": "unknown",
    "timezone": "unknown",
    "effectiveType": "unknown",
    "errorMessage": "none",
    "errorSource": "none",
}
for col, val in cat_fill.items():
    if col in df.columns:
        df[col] = df[col].fillna(val)

for col in ["clickX", "clickY", "scrollPercent"]:
    if col in df.columns:
        df[col + "_was_missing"] = df[col].isna().astype(int)
        df[col] = df[col].fillna(0)

perf_cols = ["durationMs", "domLoadTime", "fullLoadTime", "ttfb", "rtt", "downlink"]
for col in perf_cols:
    if col in df.columns:
        df.loc[df[col] < 0, col] = np.nan
        median_val = df[col].median(skipna=True)
        df[col] = df[col].fillna(median_val)

for col in ["errorLine", "errorColumn"]:
    if col in df.columns:
        df[col] = df[col].fillna(0)

In [136]:
# === Outlier clipping ===
numeric_cols = df.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    low, high = df[col].quantile([0.01, 0.99])
    df[col] = df[col].clip(lower=low, upper=high)
if "scrollPercent" in df.columns:
    df["scrollPercent"] = df["scrollPercent"].clip(0, 100)

In [137]:
# ============================================
# 🧩 Feature Engineering
# ============================================

# Split screenResolution -> width & height
if "screenResolution" in df.columns:
    def split_resolution(val):
        try:
            w, h = val.lower().replace(" ", "").split("x")
            return int(w), int(h)
        except Exception:
            return np.nan, np.nan
    wh = df["screenResolution"].apply(lambda x: pd.Series(split_resolution(str(x))))
    df["screenWidth"], df["screenHeight"] = wh[0], wh[1]
    df["aspectRatio"] = df["screenWidth"] / (df["screenHeight"] + 1e-6)
    df.drop(columns=["screenResolution"], inplace=True, errors="ignore")
    print("Parsed screenResolution -> screenWidth/screenHeight")
else:
    df["aspectRatio"] = np.nan

Parsed screenResolution -> screenWidth/screenHeight


In [138]:
# =============================
# 🔍 Feature Engineering Enhancements
# =============================

# ---- 1. Device Type from screen size ----
def classify_device(w, h):
    if pd.isna(w) or pd.isna(h):
        return "unknown"
    if w <= 480:
        return "mobile-small"
    elif w <= 768:
        return "mobile-large"
    elif w <= 1024:
        return "tablet"
    else:
        return "desktop"

df["deviceType"] = df.apply(lambda r: classify_device(r["screenWidth"], r["screenHeight"]), axis=1)

# Encode device type (low cardinality)
df["deviceType"] = df["deviceType"].astype("category").cat.codes


# ---- 2. URL-derived features ----
if "url" in df.columns:
    # Extract basic URL length and depth
    df["url_length"] = df["url"].astype(str).apply(len)
    df["url_depth"] = df["url"].astype(str).apply(lambda x: x.count("/"))

    # Try to extract domain category (if URLs are full)
    df["is_course_page"] = df["url"].astype(str).apply(lambda x: 1 if "course" in x.lower() else 0)
    df["is_dashboard"] = df["url"].astype(str).apply(lambda x: 1 if "dashboard" in x.lower() else 0)


# ---- 3. Browser / Language simplification ----
if "browser" in df.columns:
    df["browser_simplified"] = df["browser"].astype(str).str.lower()
    df["browser_simplified"] = df["browser_simplified"].replace({
        "chrome mobile": "chrome",
        "chrome android": "chrome",
        "safari mobile": "safari",
        "edge chromium": "edge",
        "firefox mobile": "firefox",
    })
    df["browser_simplified"] = df["browser_simplified"].astype("category").cat.codes

if "language" in df.columns:
    df["lang_main"] = df["language"].astype(str).apply(lambda x: x.split("-")[0].lower() if isinstance(x, str) else "unknown")
    df["lang_main"] = df["lang_main"].astype("category").cat.codes


# ---- 4. Country grouping ----
region_map = {
    "us": "americas", "ca": "americas", "mx": "americas",
    "br": "americas", "ar": "americas",
    "in": "asia", "jp": "asia", "cn": "asia", "sg": "asia",
    "uk": "europe", "de": "europe", "fr": "europe", "es": "europe",
}
df["region_group"] = df["country"].astype(str).str.lower().map(region_map).fillna("other")
df["region_group"] = df["region_group"].astype("category").cat.codes


# ---- 5. Behavioral ratios ----
if {"scrollPercent", "durationMs"}.issubset(df.columns):
    df["scroll_rate"] = df["scrollPercent"] / (df["durationMs"] + 1e-3)

if {"clickY", "screenHeight"}.issubset(df.columns):
    df["click_y_ratio"] = df["clickY"] / (df["screenHeight"] + 1e-3)

if {"ttfb", "rtt"}.issubset(df.columns):
    df["network_efficiency"] = df["ttfb"] / (df["rtt"] + 1e-3)


# ---- 6. Log-transform skewed numeric features ----
numeric_cols = df.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    if df[col].skew() > 1:
        df[col + "_log1p"] = np.log1p(df[col].clip(lower=0))


# ---- 7. Drop redundant / intermediate columns if necessary ----
drop_temp = ["url", "browser", "language", "country"]
df.drop(columns=[c for c in drop_temp if c in df.columns], inplace=True, errors="ignore")

print("\n✅ Extended feature engineering complete.")
print(df.head(3).T)


✅ Extended feature engineering complete.
                                   0                 1            2
eventType                  page_view  page_performance  page_hidden
region                       unknown           unknown      unknown
city                         unknown           unknown      unknown
timezone                     unknown           unknown      unknown
clickX                           0.0               0.0          0.0
clickY                           0.0               0.0          0.0
scrollPercent                    0.0               0.0          0.0
durationMs                   66660.0           66660.0      19514.0
domLoadTime                   1892.0            5552.0       1892.0
fullLoadTime                     0.0               0.0          0.0
ttfb                           255.0             291.0        255.0
downlink                         7.0               7.0          7.0
effectiveType                unknown           unknown      unknown
rtt   

In [139]:
# ============================================
# 🧮 Encoding Categorical Variables
# ============================================
cat_cols = df.select_dtypes(include=["object"]).columns.tolist()

# Separate low- and high-cardinality columns
low_card = [c for c in cat_cols if df[c].nunique() <= 50]
high_card = [c for c in cat_cols if df[c].nunique() > 50]

# Ordinal encode low-cardinality columns
if low_card:
    enc = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
    df[low_card] = enc.fit_transform(df[low_card])

# Target encode high-cardinality columns (like url)
if high_card:
    target_col = "durationMs"
    target_enc = TargetEncoder(cols=high_card)
    df[high_card] = target_enc.fit_transform(df[high_card], df[target_col])

In [140]:
# ============================================
# 📈 Target Transformation (optional)
# ============================================
df["durationMs_log"] = np.log1p(df["durationMs"])

In [141]:
# ============================================
# 🧹 Downcast numeric columns
# ============================================
for col in df.select_dtypes(include=["float64"]).columns:
    df[col] = pd.to_numeric(df[col], downcast="float")
for col in df.select_dtypes(include=["int64"]).columns:
    df[col] = pd.to_numeric(df[col], downcast="integer")

print("\n✅ Final schema:")
print(df.info(memory_usage="deep"))
print(f"Final memory: {memory_usage_mb(df):.2f} MB")


✅ Final schema:
<class 'pandas.core.frame.DataFrame'>
Index: 109566 entries, 0 to 134910
Data columns (total 50 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   eventType                  109566 non-null  float32
 1   region                     109566 non-null  float64
 2   city                       109566 non-null  float64
 3   timezone                   109566 non-null  float64
 4   clickX                     109566 non-null  float32
 5   clickY                     109566 non-null  float32
 6   scrollPercent              109566 non-null  float32
 7   durationMs                 109566 non-null  float64
 8   domLoadTime                109566 non-null  float32
 9   fullLoadTime               109566 non-null  float32
 10  ttfb                       109566 non-null  float32
 11  downlink                   109566 non-null  float32
 12  effectiveType              109566 non-null  float32
 13  rtt              

In [142]:
# ============================================
# 💾 Save cleaned dataset
# ============================================
output_path = "cleaned_advanced_events.csv"
df.to_csv(output_path, index=False)
print(f"\nSaved optimized dataset → {output_path}")


Saved optimized dataset → cleaned_advanced_events.csv


In [143]:
print(df.head(5))

   eventType         region           city       timezone  clickX  clickY  \
0        6.0  472902.518434  472902.518434  442593.412567     0.0     0.0   
1        5.0  472902.518434  472902.518434  442593.412567     0.0     0.0   
2        3.0  472902.518434  472902.518434  442593.412567     0.0     0.0   
3        6.0  472902.518434  472902.518434  442593.412567     0.0     0.0   
4        5.0  472902.518434  472902.518434  442593.412567     0.0     0.0   

   scrollPercent  durationMs  domLoadTime  fullLoadTime  ...  ttfb_log1p  \
0            0.0     66660.0       1892.0           0.0  ...    5.545177   
1            0.0     66660.0       5552.0           0.0  ...    5.676754   
2            0.0     19514.0       1892.0           0.0  ...    5.545177   
3            0.0     66660.0       1892.0           0.0  ...    5.545177   
4            0.0     66660.0       4433.0           0.0  ...    5.267858   

   screenHeight_log1p  url_length_log1p  url_depth_log1p  is_dashboard_log1p  \
